In [ ]:
import polars as pl
import networkx as nx
import rtsvg
rt = rtsvg.RACETrack()
from linknode_graph_patterns import LinkNodeGraphPatterns

def graphToDataFrame(_g_):
    _lu_ = {'fm':[], 'to':[]}
    for _e_ in _g_.edges: _lu_['fm'].append(_e_[0]), _lu_['to'].append(_e_[1])
    return pl.DataFrame(_lu_)
_dfs_, _patterns_, _node_colors_ = [], LinkNodeGraphPatterns(), {}
for _pattern_ in _patterns_.types:
    _g_  = _patterns_.createPattern(_pattern_, prefix=_pattern_+'__')
    for _node_ in _g_.nodes: _node_colors_[_node_] = _pattern_
    _dfs_.append(graphToDataFrame(_g_))
df    = pl.concat(_dfs_)
g     = rt.createNetworkXGraph(df, [('fm','to')])
_pos_ = nx.spring_layout(g) # rtsvg.PolarsForceDirectedLayout(g).results()

In [ ]:
pos_adj, shapes_gen = rt.circlePackGraphComponentPlacement(g, _pos_)
pos_tm              = rt.treeMapGraphComponentPlacement(g, _pos_)
_params_ = {'df':df, 'relationships':[('fm','to')], 'w':512, 'h':512, 'node_size':3, 'node_color':_node_colors_}
rt.tile([rt.link(**_params_, pos=_pos_),rt.link(**_params_, pos=pos_adj, bg_shape_lu=shapes_gen), rt.link(**_params_, pos=pos_tm)])

In [ ]:
#_igl_ = rt.interactiveGraphPanel(df, ln_params={'relationships': [('fm','to')], 'pos':pos_adj})
#_igl_

In [ ]:
#
# Claude Sonnet 4.5:  Python code for "i have n points as a list of (x,y) tuples.  I need the smallest possible circle the inscribes all of the points."
#
import math
import random

def smallest_enclosing_circle(points):
    """
    Find the smallest circle that encloses all given points.
    Uses Welzl's algorithm with expected O(n) time complexity.
    
    Args:
        points: List of (x, y) tuples
    
    Returns:
        Tuple of ((cx, cy), radius) where (cx, cy) is circle center
    """
    # Shuffle for better average performance
    shuffled = points.copy()
    random.shuffle(shuffled)
    return _welzl_helper(shuffled, [], len(shuffled))

def _welzl_helper(P, R, n):
    """Recursive helper for Welzl's algorithm"""
    if n == 0 or len(R) == 3:
        return _min_circle_trivial(R)
    
    # Pick a random point
    p = P[n - 1]
    
    # Get circle without this point
    circle = _welzl_helper(P, R, n - 1)
    
    # If point is inside circle, return it
    if circle and _is_inside(p, circle):
        return circle
    
    # Otherwise, point must be on boundary
    return _welzl_helper(P, R + [p], n - 1)

def _min_circle_trivial(points):
    """Handle base cases with 0-3 points"""
    if len(points) == 0:
        return ((0, 0), 0)
    elif len(points) == 1:
        return (points[0], 0)
    elif len(points) == 2:
        p1, p2 = points
        cx = (p1[0] + p2[0]) / 2
        cy = (p1[1] + p2[1]) / 2
        r = math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2) / 2
        return ((cx, cy), r)
    else:  # 3 points
        # Try circle from each pair
        c1 = _circle_from_2_points(points[0], points[1])
        if all(_is_inside(p, c1) for p in points):
            return c1
        
        c2 = _circle_from_2_points(points[0], points[2])
        if all(_is_inside(p, c2) for p in points):
            return c2
        
        c3 = _circle_from_2_points(points[1], points[2])
        if all(_is_inside(p, c3) for p in points):
            return c3
        
        # Need circumcircle of all 3 points
        return _circle_from_3_points(points[0], points[1], points[2])

def _circle_from_2_points(p1, p2):
    """Circle with diameter from p1 to p2"""
    cx = (p1[0] + p2[0]) / 2
    cy = (p1[1] + p2[1]) / 2
    r = math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2) / 2
    return ((cx, cy), r)

def _circle_from_3_points(p1, p2, p3):
    """Circumcircle of 3 points"""
    ax, ay = p1
    bx, by = p2
    cx, cy = p3
    
    d = 2 * (ax * (by - cy) + bx * (cy - ay) + cx * (ay - by))
    if abs(d) < 1e-10:
        # Points are collinear, return circle from furthest pair
        return _furthest_pair_circle([p1, p2, p3])
    
    ux = ((ax**2 + ay**2) * (by - cy) + (bx**2 + by**2) * (cy - ay) + 
          (cx**2 + cy**2) * (ay - by)) / d
    uy = ((ax**2 + ay**2) * (cx - bx) + (bx**2 + by**2) * (ax - cx) + 
          (cx**2 + cy**2) * (bx - ax)) / d
    
    r = math.sqrt((ax - ux)**2 + (ay - uy)**2)
    return ((ux, uy), r)

def _furthest_pair_circle(points):
    """Circle from furthest pair of points"""
    max_dist = 0
    pair = (points[0], points[1])
    for i in range(len(points)):
        for j in range(i + 1, len(points)):
            dist = math.sqrt((points[i][0] - points[j][0])**2 + 
                           (points[i][1] - points[j][1])**2)
            if dist > max_dist:
                max_dist = dist
                pair = (points[i], points[j])
    return _circle_from_2_points(pair[0], pair[1])

def _is_inside(point, circle):
    """Check if point is inside or on circle boundary"""
    (cx, cy), r = circle
    dist = math.sqrt((point[0] - cx)**2 + (point[1] - cy)**2)
    return dist <= r + 1e-10  # Small epsilon for floating point

# Example usage
if __name__ == "__main__":
    points = [(0, 0), (1, 0), (0, 1), (1, 1), (0.5, 0.5)]
    
    (center, radius) = smallest_enclosing_circle(points)
    print(f"Center: ({center[0]:.4f}, {center[1]:.4f})")
    print(f"Radius: {radius:.4f}")
    
    # Verify all points are inside
    print("\nVerifying all points are enclosed:")
    for p in points:
        dist = math.sqrt((p[0] - center[0])**2 + (p[1] - center[1])**2)
        print(f"  Point {p}: distance = {dist:.4f} (inside: {dist <= radius + 1e-10})")

In [ ]:
import time
_perf_ = {'points': [], 'time': []}
for n in range(100, 10_000, 500):
    points = [(random.uniform(-100, 100), random.uniform(-100, 100)) for _ in range(n)]
    t0 = time.time()
    (center, radius) = smallest_enclosing_circle(points)
    t1 = time.time()
    _perf_['points'].append(n), _perf_['time'].append(t1 - t0)